In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [84]:
from datetime import datetime
from io import StringIO
import re

In [2]:
url = r'https://en.wikipedia.org/wiki/List_of_American_films_of_2021'
r = requests.get(url)
r.status_code

200

In [3]:
soup1 = BeautifulSoup(r.content, 'html.parser')
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

In [87]:
tables = soup2.find_all('table')

In [135]:
title_link = []
for_range = len(tables[3].find_all('tr')) -1

for i in range(for_range):
    i += 1
    messy = tables[3].find_all('tr')[i].find_all('td')[-4]
    title_link.append(messy.find('a'))

title_link[0]

<a href="/wiki/Shadow_in_the_Cloud" title="Shadow in the Cloud">
                Shadow in the Cloud
               </a>

In [137]:
title_link = list(map(str, title_link))

In [139]:
link = []
title = []

pattern = r'="(.+)"\s'
pattern2 = r'e="(.+)"'

for i in title_link:
    link.append(re.findall(pattern, i)[0])
    title.append(re.findall(pattern2, i)[0])

link[:5]

['/wiki/Shadow_in_the_Cloud',
 '/wiki/Hacksaw_(film)',
 '/wiki/Dr._Bird%27s_Advice_for_Sad_Poets',
 '/wiki/The_White_Tiger_(2021_film)',
 '/wiki/Locked_Down_(film)']

In [142]:
df2 = pd.DataFrame({'Link':link, 'Title':title})
df2.tail()

,Link,Title
63,/wiki/Nobody_(2021_film),Nobody (2021 film)
64,/wiki/Bad_Trip_(film),Bad Trip (film)
65,/wiki/The_Seventh_Day_(2021_film),The Seventh Day (2021 film)
66,/wiki/Senior_Moment,Senior Moment
67,/wiki/Godzilla_vs._Kong,Godzilla vs. Kong


In [140]:
df = pd.read_html(StringIO(str(tables[3])))
df[0].tail()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
63,M A R C H,26,Nobody,Universal Pictures / Perfect World Pictures ...,Ilya Naishuller (director); Derek Kolstad (...,[54]
64,M A R C H,26,Bad Trip,Netflix / Orion Pictures,Kitao Sakurai (director/screenplay); Dan Curry...,NaN
65,M A R C H,26,The Seventh Day,Vertical Entertainment / Voltage Pictures,Justin P. Lange (director/screenplay); Guy Pe...,[55]
66,M A R C H,26,Senior Moment,Screen Media Films,"Giorgio Serafini (director); Kurt Brungardt, C...",[56]
67,M A R C H,31,Godzilla vs. Kong,Warner Bros. Pictures / Legendary Entertainm...,"Adam Wingard (director); Eric Pearson , Ma...",[57]


In [145]:
df = pd.merge(df2, df[0], on='Title')
df.head()

,Link,Title,Opening,Opening.1,Production company,Cast and crew,Ref.
0,/wiki/Shadow_in_the_Cloud,Shadow in the Cloud,J A N U A R Y,1,Vertical Entertainment,Roseanne Liang (director/screenplay); Max La...,[2]
1,/wiki/Dr._Bird%27s_Advice_for_Sad_Poets,Dr. Bird's Advice for Sad Poets,J A N U A R Y,12,Relativity Media / Ketchup Entertainment,Yaniv Raz (director/screenplay); Lucas Jade Z...,[4]
2,/wiki/Outside_the_Wire,Outside the Wire,J A N U A R Y,15,Netflix,"Mikael Håfström (director); Rob Yescombe, Row...",[7]
3,/wiki/The_Ultimate_Playlist_of_Noise,The Ultimate Playlist of Noise,J A N U A R Y,15,Hulu,Bennett Lasseter (director); Mitchell Winkie (...,[9]
4,/wiki/Our_Friend,Our Friend,J A N U A R Y,22,Gravitas Ventures / Black Bear Pictures,Gabriela Cowperthwaite (director); Brad Inge...,[12]


In [148]:
months = {'January':1, 'February':2, 'March':3,
          'April':4, 'May':5, 'June':6, 'July':7,
          'August':8,'September':9, 'October':10,
          'November':11, 'December':12}

In [149]:
def get_number(x):
    return months[x]

df['Opening'] = df['Opening'].str.replace(' ', '').str.capitalize()
df['Opening'] = df['Opening'].apply(get_number)
df.head()

,Link,Title,Opening,Opening.1,Production company,Cast and crew,Ref.
0,/wiki/Shadow_in_the_Cloud,Shadow in the Cloud,1,1,Vertical Entertainment,Roseanne Liang (director/screenplay); Max La...,[2]
1,/wiki/Dr._Bird%27s_Advice_for_Sad_Poets,Dr. Bird's Advice for Sad Poets,1,12,Relativity Media / Ketchup Entertainment,Yaniv Raz (director/screenplay); Lucas Jade Z...,[4]
2,/wiki/Outside_the_Wire,Outside the Wire,1,15,Netflix,"Mikael Håfström (director); Rob Yescombe, Row...",[7]
3,/wiki/The_Ultimate_Playlist_of_Noise,The Ultimate Playlist of Noise,1,15,Hulu,Bennett Lasseter (director); Mitchell Winkie (...,[9]
4,/wiki/Our_Friend,Our Friend,1,22,Gravitas Ventures / Black Bear Pictures,Gabriela Cowperthwaite (director); Brad Inge...,[12]


In [150]:
df.rename(columns={'Opening':'Month', 'Opening.1':'Day'}, inplace=True)

df = df[['Title', 'Month', 'Day', 'Link']]

In [151]:
df['Year'] = 2021

In [176]:
r2 = requests.get(r'https://en.wikipedia.org/wiki/Shadow_in_the_Cloud')
print(r2.status_code)
soup1_2 = BeautifulSoup(r2.content, 'html.parser')

200


In [177]:
soup1_2.find_all('p')[0].get_text(strip=False)

'Shadow in the Cloud is a 2020 action horror film directed by Roseanne Liang, from a screenplay by Liang and Max Landis,[4] starring Chloë Grace Moretz, Beulah Koale, Taylor John Smith, Callan Mulvey and Nick Robinson. It follows a female flight officer on a top-secret mission in the Pacific during the Second World War, who after boarding a Boeing B-17 Flying Fortress, encounters an evil gremlin during the flight.\n'

In [188]:
pattern3 = r'[2021,2020] [A-Z]\w+([^\.]+) film'
pattern4 = r'[2021,2020] ([^\.]+) film'

first_line = soup1_2.find_all('p')[0].get_text(strip=False)
re.findall(pattern4, first_line)

['action horror']